<a href="https://colab.research.google.com/github/m10923039/ML/blob/main/VGG16_best.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import os
# tf.keras.backend.clear_session()
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)
# print("All devices: ", tf.config.list_logical_devices('TPU'))
# strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [ ]:
# from zipfile import ZipFile
# z = ZipFile('/content/drive/MyDrive/Colab Notebooks/hw02/cifar10/data/train.zip', mode='r')
# z.extractall('/content/drive/MyDrive/Colab Notebooks/hw02/cifar10/data')

# 開始

In [ ]:
import os
import cv2 
import numpy as np
from PIL import Image
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout,Conv2D,Activation,BatchNormalization,MaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plt
import numpy as np
import cv2


# 讀取檔案 


*   設定 training dataset 和 validation dataset
*   validation_split=0.2 validation的比例
*   subset='training' 設定產生的是 training dataset 或 validation dataset
*   seed=0  training 和 validation 的值一定要一樣，確保資料部會重複



In [ ]:
train_dir='/content/drive/MyDrive/Colab Notebooks/hw02/cifar10/data/train'
train_dataset=tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=64,
    image_size=(32, 32),
    shuffle=True,
    validation_split=0.2,
    subset='training',
    seed=0,
    interpolation="bilinear"
)
val_dataset=tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=64,
    image_size=(32, 32),
    shuffle=True,
    validation_split=0.2,
    subset='validation',
    seed=0,
    interpolation="bilinear"
)

Found 50000 files belonging to 10 classes.
Using 40000 files for training.
Found 50000 files belonging to 10 classes.
Using 10000 files for validation.


# min max normalization 0～255 轉 0～1




In [ ]:
from tensorflow.keras import layers
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), tf.cast(y, dtype=tf.int32)))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), tf.cast(y, dtype=tf.int32)))

In [ ]:
l2 = tf.keras.regularizers.l2(0.01)
params = {'activation': 'relu',
      'optimizer': 'Adam',
      'losses': 'categorical_crossentropy',
      'batch_size': 1500,
      'epochs': 60,
      'node_1' : 384,
      'node_2' : 256,
      'regularizers' :l2,
      'input_shape' :(32,32,3),
      'output_shape' : 10
    }
def create_model(params):
      net = VGG16(include_top=False, weights='imagenet', input_tensor=None,
              input_shape = (params['input_shape'])
              )
      
      # 透過layer_name 節取 model的某一層 block3_pool 
      layer_name='block3_pool'

      # 設定取到該層的都使用預設參數，不訓練參數
      layer_index=net.layers.index(net.get_layer(layer_name))
      
      # 建立模型
      base_model = net.get_layer(layer_name).output
  # base_model = tf.keras.layers.Conv2D(128, (1, 1), padding='valid')(base_model)
  # base_model = tf.keras.layers.MaxPooling2D( pool_size = (3, 3) )(base_model)
      # 攤平 feature map 
      base_model = Flatten()(base_model)

      base_model = Dropout(0.3)(base_model)
      base_model = tf.keras.layers.BatchNormalization()(base_model)
      base_model = Dense(units = params['node_1'], kernel_initializer = tf.keras.initializers.GlorotNormal()
                 ,kernel_regularizer= params['regularizers'] , activation= params['activation'])(base_model)


      base_model = Dense(units = params['node_2'], kernel_initializer = tf.keras.initializers.GlorotNormal()
                 ,kernel_regularizer = params['regularizers'] , activation= params['activation'])(base_model)

      base_model = Dropout(0.3)(base_model)
      output_layer = Dense(params['output_shape'], activation='softmax', name='softmax')(base_model)

      # 設定凍結與要進行訓練的網路層
      net_final = Model(inputs=net.input, outputs=output_layer)
      for layer in net_final.layers[:layer_index]:
          layer.trainable = False

      return net_final

In [ ]:
# with strategy.scope():
model = create_model(params)
precision=tf.keras.metrics.Precision(name='precision')
recall=tf.keras.metrics.Recall(name='recall')
accuracy=tf.keras.metrics.CategoricalAccuracy(name='accuracy')
model.compile(optimizer=params['optimizer'],
        loss=params['losses'],
        metrics = [recall,precision,accuracy],
        
        )
print(model.summary())
callbacks_list=[
  tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',mode='min', patience=20 ,restore_best_weights=True),
  tf.keras.callbacks.ModelCheckpoint(
            filepath='/content/drive/MyDrive/Colab Notebooks/hw02/cifar10/C10_VGG16_01',
            monitor='val_loss',
            save_best_only=True),
  tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                        patience=5,                      
                        verbose=1,
                        factor=0.3,  
                        min_lr=0.00001 )  
        ]
out = model.fit(
          train_dataset,
          validation_data=val_dataset,
          epochs=100,
          callbacks=callbacks_list               
            )

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0   

# 訓練 驗證 績效圖

In [ ]:
import matplotlib.pyplot as plot  # plot 可以視為畫布

def train_history_graphic( history       # 資料集合
                         , history_key1  # 資料集合裡面的來源 1 (有 loss, acc, val_loss, val_acc 四種)
                         , history_key2  # 資料集合裡面的來源 2 (有 loss, acc, val_loss, val_acc 四種)
                         , y_label       # Y 軸標籤文字
                         , index=0
                         ) :
    plot.figure(figsize=(15,10))
    font_size=25
    # 資料來源 1
    plot.plot( history.history[history_key1][index:] )
    # 資料來源 2
    plot.plot( history.history[history_key2][index:] )
    # 標題
    plot.title( 'train history',fontsize=font_size )
    # X 軸標籤文字
    plot.xlabel( 'epochs',fontsize=font_size )
    # Y 軸標籤文字
    plot.ylabel( y_label,fontsize=font_size )
    # 設定圖例
    # (參數 1 為圖例說明, 有幾個資料來源, 就對應幾個圖例說明)
    # (參數 2 為圖例位置, upper 為上面, lower 為下面, left 為左邊, right 為右邊)
    plot.legend( ['train', 'validate']
               , loc = 'upper left'
               )
    # 顯示畫布
    plot.show()
# 

#############################################
metrics_list=['loss','recall','precision','accuracy']
for metrics in metrics_list :
  train_history_graphic( vgg1, metrics, f'val_{metrics}', metrics, 0 )


# precision,recall,accuracy 計算

In [ ]:
from tensorflow.keras import layers
test_dir='/content/drive/MyDrive/Colab Notebooks/hw02/cifar10/data/test'
test_dataset=tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=64,
    image_size=(32, 32),
    shuffle=True,
    interpolation="bilinear"
)
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), tf.cast(y, dtype=tf.int32)))

Found 9589 files belonging to 10 classes.


In [ ]:
vgg1=tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/hw02/cifar10/C10_VGG16_01')

In [ ]:
# precision=tf.keras.metrics.Precision(name='precision')
# recall=tf.keras.metrics.Recall(name='recall')
# accuracy=tf.keras.metrics.CategoricalAccuracy(name='accuracy')
# vgg1.compile(optimizer='Adam',
#     loss='categorical_crossentropy',
#     metrics = [precision,recall,accuracy]
#           )


In [ ]:
preds=vgg1.evaluate(test_dataset)

 21/150 [===>..........................] - ETA: 1:03:51 - loss: 0.7398 - recall: 0.7426 - precision: 0.8408 - accuracy: 0.7879

In [ ]:
def evaluate_model_1(model,test_dataset):
  precision=tf.keras.metrics.Precision(name='precision')
  recall=tf.keras.metrics.Recall(name='recall')
  accuracy=tf.keras.metrics.CategoricalAccuracy(name='accuracy')
  model.compile(optimizer='Adam',
        loss='categorical_crossentropy',
        metrics = [precision,recall,accuracy]
              )
  preds=model.evaluate(test_dataset)
  print(model)
  print('loss ： %.5f' % preds[0])
  print('precision：%.5f' % preds[1])
  print('recall：%.5f' % preds[2])
  print('accuracy： %.5f' %preds[3])
  F1 = 2 * (preds[1] * preds[2]) / (preds[1] + preds[2])
  print('f1score：%.5f' % F1)
  print("===========================================")
  preds.append(F1)
  return pd.DataFrame(preds,columns=['loss','precision','recall','accuracy','f1score'])

# model out 
df=evaluate_model_1(vgg1,test_dataset)
df